## PPI General Linear Model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels as sm
import pandas as pd
import numpy as np
import os

from bids import BIDSLayout
from scipy import io

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.glm_utils import convolve, Regressor

In [ ]:
# Directory for PPI analysis
path_out = os.path.join(path_root, 'data/main_fmri_study/derivatives/ppi')
path_timeries = os.path.join(path_out, 'timeseries')

# Load behavioral data
path_beh = os.path.join(path_root, 'data/main_fmri_study/sourcedata/behavioral')
beh, meta = load_behavioral_data(path=path_beh, verbose=True)
n_subjects, n_conditions, n_trials, _ = beh.shape
n_volumes = 730
t_r = 2

# Load trial modulations
path_modulations = os.path.join(path_root, 'data/main_fmri_study/derivatives/nistats/modulations')
modulations_perr = np.load(os.path.join(path_modulations, 'modulations_perr.npy'))

# Load neural & BOLD timeseries
data = io.loadmat(os.path.join(path_timeries, 'time_series_neural_all.mat'))
time_series_neural_all = data['time_series_neural_all']
time_series_clean_all = np.load(os.path.join(path_timeries, 'time_series_clean_all.npy'))
downsamples = data['k'].flatten()

In [ ]:
sub = 0
con = 0

idx_seed = 0
idx_target = 1

#### Create design matrix

In [ ]:
def upsampled_events(onsets, n_volumes, duration, t_r, sampling_rate=1/16):
    '''...'''
    n_frames_per_event = int(duration / (t_r * sampling_rate))
    frame_times_up = np.arange(0, n_volumes*t_r, t_r*sampling_rate)
    ts_event_up = np.zeros(frame_times_up.shape)

    for event in onsets:
        first_frame = np.argmax(frame_times_up > event)
        ts_event_up[range(first_frame, first_frame + n_frames_per_event)] = 1
        
    return ts_event_up - ts_event_up.mean()

In [ ]:
sampling_rate = 1/16
onset_out = beh[sub, con, :, meta['dim4'].index('onset_out')]
frame_times = np.arange(n_volumes) * t_r

y = time_series_clean_all[sub, con, :, idx_target]

reg_physio = time_series_clean_all[sub, con, :, idx_seed]
reg_psycho = Regressor(
    name='prediction error', 
    frame_times=frame_times,
    duration=np.ones(n_trials) * t_r,
    onset=onset_out,
    modulation=modulations_perr[sub, con]
).dm_column

ts_neural_up = time_series_neural_all[sub, con, :, idx_seed]
ts_event_up = upsampled_events(onset_out, n_volumes=n_volumes, duration=t_r, t_r=t_r)
ts_ppi_up = ts_neural_up * ts_event_up

reg_ppi = convolve(ts_ppi_up, t_r=t_r*sampling_rate)[downsamples]